使用券商API下載股票資訊

In [7]:
import os 
import shioaji as sj
from dotenv import load_dotenv

load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))


Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


[FutureAccount(person_id='F229985348', broker_id='F002000', account_id='1553688', username='黃子芮'),
 StockAccount(person_id='F229985348', broker_id='9A89', account_id='2160122', signed=True, username='黃子芮')]

In [8]:
api.Contracts.Stocks['2330']

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2330', symbol='TSE2330', name='台積電', category='24', unit=1000, limit_up=1035.0, limit_down=849.0, reference=943.0, update_date='2024/08/16', margin_trading_balance=345, day_trade=<DayTrade.Yes: 'Yes'>)

In [9]:
import sqlite3
import pandas as pd

def get_stock(code, connection, api):

    try:
        sql = "SELECT * FROM stocks WHERE code ='{}' ".format(code)
        df = pd.read_sql(sql, connection, index_col=['code'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    stock = api.Contracts.Stocks[code]

    stock_dict = {
        'code': [stock.code],
        'name': [stock.name],
        'category': [stock.category],
        'day_trade': [stock.day_trade.value]
    }

    df = pd.DataFrame(data = stock_dict)
    df = df.set_index('code')

    return df, False

In [10]:
connection = sqlite3.connect('data.db')

df, in_db= get_stock('2330', connection, api)

print(in_db)
df

False


,name,category,day_trade
code,,,
2330,台積電,24,Yes


CH2-6 API 下載股票資訊與資料庫整合實作

In [11]:
from exchange_calendars import get_calendar
import time

def update_stocks(daily_target, connection, api):

    main_df = pd.DataFrame()

    code_list =[]

    for codes in daily_target.values():
        code_list.extend(codes)

    code_list = list(set(code_list))

    for code in code_list:
        df, in_db = get_stock(code, connection, api)

        if df is not None and not in_db:
            main_df = pd.concat([main_df,df], sort = False)
            time.sleep(1)
        
    if not main_df.empty:
        main_df.to_sql('stocks', connection, if_exists='append')
        return main_df


In [12]:
daily_target ={
    pd.to_datetime('2024-07-29'): ['2303', '2603', '2609', '3576', '6443'],
    pd.to_datetime('2024-08-01'): ['2324', '3576', '6443'],
    pd.to_datetime('2024-08-02'): ['1609', '1714', '1802', '2312', '2344', '2349',
                                   '2409', '2603', '3481', '3576', '3704', '6116', '6191']
                }
daily_target

{Timestamp('2024-07-29 00:00:00'): ['2303', '2603', '2609', '3576', '6443'],
 Timestamp('2024-08-01 00:00:00'): ['2324', '3576', '6443'],
 Timestamp('2024-08-02 00:00:00'): ['1609',
  '1714',
  '1802',
  '2312',
  '2344',
  '2349',
  '2409',
  '2603',
  '3481',
  '3576',
  '3704',
  '6116',
  '6191']}

登入帳號

In [13]:
import os 
import shioaji as sj
from dotenv import load_dotenv

load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'), os.getenv('SECRET_KEY'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [14]:
update_stocks(daily_target, connection, api)

In [15]:
get_stock('2330', connection, api)     #找出單一股票欄位
get_stock('2330', connection, api)[0]      #只取Dataframe
#get_stock('2303', connection, api)[0].iloc[0]['day_trade']     #第一筆欄位中的Daytrade欄位
#get_stock('2303', connection, api)[0].iloc[0]['day_trade'] == 'Yes'     #用可否當沖來篩選股票

,name,category,day_trade
code,,,
2330,台積電,24,Yes


篩選可以當沖的股票

In [16]:
codes =  ['2303','2603', '2609', '3576', '6443']
day_trading_codes = [code for code in codes if get_stock(code,connection,api)[0].iloc[0]['day_trade'] == 'Yes']
day_trading_codes

['2303', '2603', '2609', '3576', '6443']

2-7 抓取逐筆成交資料

In [32]:
import time
def get_ticks(code, date, connection, api):

    try:    
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                 date,
                                                                                 date+datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    ticks = api.ticks(api.Contracts.Stocks[code], date.strftime('%Y-%m-%d'))
    df = pd.DataFrame({**ticks})

    df.ts = pd.to_datetime(df.ts)
    df['code'] = code
    df = df.set_index('ts')

    return df, False

In [36]:
df, in_db = get_ticks('2330', pd.to_datetime('2024/8/1'), connection, api)

print(in_db)
df

True


,tick_type,ask_volume,close,bid_price,bid_volume,volume,ask_price,code
ts,,,,,,,,
2024-08-01 09:00:01.867335,1,32,969.0,968.0,80,4386,969.0,2330
2024-08-01 09:00:01.908387,2,32,968.0,968.0,64,16,969.0,2330
2024-08-01 09:00:01.908690,2,32,968.0,968.0,48,16,969.0,2330
2024-08-01 09:00:01.908730,2,32,968.0,968.0,47,1,969.0,2330
2024-08-01 09:00:01.909599,2,28,968.0,968.0,43,4,969.0,2330
...,...,...,...,...,...,...,...,...
2024-08-01 13:24:57.358001,1,118,959.0,958.0,46,2,959.0,2330
2024-08-01 13:24:57.837681,2,117,958.0,958.0,46,1,959.0,2330
2024-08-01 13:24:58.243640,1,116,959.0,958.0,46,1,959.0,2330


In [38]:
def update_ticks(daily_target, connection, api):
    main_df = pd.DataFrame()

    tw_calendar = get_calendar('XTAI')

    for date, codes in daily_target.items():

        day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']

        print('正在更新{}逐筆成交資料，總共{}檔標的，為{}'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes), day_trading_codes))

        for code in day_trading_codes:

            df, in_db = get_ticks(code, date, connection, api)

            if df is not None and in_db:
                main_df = pd.concat([main_df, df], sort = False)
                time.sleep(1)

            prev_trading_date = tw_calendar.previous_close(date).date()
            prev_df, prev_in_db = get_ticks(code, prev_trading_date, connection, api)

            if prev_df is not None and not prev_in_db:
                main_df = pd.concat([main_df, prev_df], sort = False)
                time.sleep(1)
    
    if not main_df.empty:
        main_df.to_sql('ticks', connection, if_exists='append')
        return main_df



In [44]:
daily_targets ={
    pd.to_datetime('2024-08-02'): ['2317', '006208', '2382', '3189', '3714','2330'],
    pd.to_datetime('2024-08-05'): ['2317', '2330', '6188'],
    pd.to_datetime('2024-08-06'): ['2317', '4938', '6188', '3189', '2834', '2330', '3037']
                }

In [45]:
update_ticks(daily_targets, connection, api)

正在更新2024/08/02逐筆成交資料，總共6檔標的，為['2317', '006208', '2382', '3189', '3714', '2330']
正在更新2024/08/05逐筆成交資料，總共3檔標的，為['2317', '2330', '6188']
正在更新2024/08/06逐筆成交資料，總共6檔標的，為['2317', '4938', '6188', '3189', '2330', '3037']


,tick_type,ask_volume,close,bid_price,bid_volume,volume,ask_price,code
ts,,,,,,,,
2024-08-01 09:00:00.462169,1,538,205.0,205.0,158,5150,205.5,2317
2024-08-01 09:00:00.500406,1,537,205.5,205.0,158,1,205.5,2317
2024-08-01 09:00:00.542133,1,536,205.5,205.0,158,1,205.5,2317
2024-08-01 09:00:00.542719,1,534,205.5,205.0,158,2,205.5,2317
2024-08-01 09:00:00.566921,2,534,205.0,205.0,126,32,205.5,2317
...,...,...,...,...,...,...,...,...
2024-08-05 13:24:41.448736,2,914,149.5,0.0,0,2,0.0,3037
2024-08-05 13:24:51.536636,2,912,149.5,0.0,0,1,0.0,3037
2024-08-05 13:24:56.672441,2,911,149.5,0.0,0,1,0.0,3037
